In [ ]:
# Importing libraries

import snappy                     # SNAP python interface
from snappy import GPF, jpy       # jpy is Pyhton-Java bridge
import numpy as np                # scientific computing
from osgeo import gdal            
import os                         # data access in file manager
import glob                       # data access in file manager
import datetime                 
import pandas as pd               # data analysis and manipulation
import matplotlib.pyplot as plt   # create visualizations
from zipfile import ZipFile
from sentinel1_lake_extraction import sentinel1_lake_extraction as le

from IPython.core.display import Image, display

display(Image(r"/home/baris/Desktop/CE-STAR/papers_figures_reports/raporlar/ProcessWorkFlow.png", width=1900, unconfined=False))
display(Image(r"/home/baris/Desktop/CE-STAR/papers_figures_reports/raporlar/Example.png", width=1500, unconfined=False))
display(Image(url= "https://www.meccanismocomplesso.org/wp-content/uploads/2017/04/OpenCV-Otsu-Binary-threshold-of-noisy_leaf-image-2-1.jpg", width=500, unconfined=False))


##
sentinel1_lake = le()



In [ ]:
def output_view(product, band, vv_min,vv_max,vh_min,vh_max):
    '''Create visualization of processed Sentinel-1 SAR data'''
    

    band_data_list = []
    for prd in product:
        for i in band:
            band = prd.getBand(i)
            w = band.getRasterWidth()
            h = band.getRasterHeight()
            band_data = np.zeros( w * h , np.float32)
            band.readPixels(0, 0, w, h, band_data)
            band_data.shape = h, w
            band_data_list.append(band_data)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize= (16,16))
    ax1.imshow(band_data_list[0], cmap = 'gray',vmin = vv_min, vmax = vv_max)
    ax1.set_title("VV polarization")
    ax2.imshow(band_data_list[1], cmap = 'gray', vmin = vh_min, vmax = vh_max)
    ax2.set_title("VH polarization")
    plt.show()
    
    
    

In [ ]:
# Finding and reading the data

sentinel1_lake.traverse_data()

product = sentinel1_lake.read_data()



In [ ]:
sentinel1_lake.calibration()
sentinel1_lake.linear_to_dB()
product = sentinel1_lake.geo_subset()           # change lake name from sentinel1_lake_extraction.py, accordingly.
band = ['Sigma0_VV_db', 'Sigma0_VH_db']
output_view(product, band, -24.07,-6.69,-27.51,-15.14)




In [ ]:
product = sentinel1_lake.terrain_correction()    #change projection system from sentinel1_lake_extraction.py
band = ['Sigma0_VV_db', 'Sigma0_VH_db']
output_view(product, band, -24.07,-6.69,-27.51,-15.14)

In [ ]:
product = sentinel1_lake.speckle_filter()
output_view(product, band, -19.53,-5.29,-25.03,-12.52)

In [ ]:
product = sentinel1_lake.apply_threshold_to_product()

def output_view_binary(product):
    '''Create visualization of processed Sentinel-1 SAR data'''
 
  
    for prd in product:
        band = prd.getBand('band1')
        w = band.getRasterWidth()
        h = band.getRasterHeight()
        binary_data = np.zeros( w * h)
        band.readPixels(0, 0, w, h, binary_data)
        binary_data.shape = h, w

    
    fig, ax1 = plt.subplots(1, figsize= (20,20))
    ax1.imshow(binary_data, cmap = 'gray',vmin = 0, vmax = 1)
    ax1.set_title("Binary image of VV polarization")
    


output_view_binary(product)

In [ ]:
product = sentinel1_lake.binary_filter()

output_view_binary(product)

